In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV': 'MEDV'}, inplace = True)

In [5]:
data.to_sql("boston", con, index = False, if_exists = 'replace')

In [6]:
pd.read_sql(
"""
select * from boston
""",
con)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
pd.read_sql(
"""
select 
    count(case when CRIM is null then 1 end) as CRIM,
    count(case when ZN is null then 1 end) as ZN,
    count(case when INDUS is null then 1 end) as INDUS,
    count(case when CHAS is null then 1 end) as CHAS,
    count(case when NOX is null then 1 end) as NOX  
from boston
""",
con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
pd.read_sql(
"""
select 
    count(distinct CRIM) CRIM,
    count(distinct ZN) ZN,
    count(distinct INDUS) INDUS,
    count(distinct CHAS) CHAS,
    count(distinct NOX) NOX
from boston
""",
con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
pd.read_sql(
"""
with minmedian as (
select
    case when min(CRIM) = (select CRIM
                                from boston
                                order by CRIM 
                                limit 1
                                offset (select count(*)/ 2 from boston)) then 'CRIM' else 'NO' end equal_median_min
from boston
union all
select
    case when min(ZN) = (select ZN
                                from boston
                                order by ZN 
                                limit 1
                                offset (select count(*)/ 2 from boston)) then 'ZN' else 'NO' end equal_median_min
from boston
union all
select
    case when min(INDUS) = (select INDUS
                                from boston
                                order by INDUS 
                                limit 1
                                offset (select count(*)/ 2 from boston)) then 'INDUS' else 'NO' end equal_median_min
from boston
union all
select
    case when min(CHAS) = (select CHAS
                                from boston
                                order by CHAS 
                                limit 1
                                offset (select count(*)/ 2 from boston)) then 'CHAS' else 'NO' end equal_median_min
from boston
union all
select
    case when min(NOX) = (select NOX
                                from boston
                                order by NOX 
                                limit 1
                                offset (select count(*)/ 2 from boston)) then 'NOX' else 'NO' end equal_median_min
from boston)
select
    *
from minmedian
where equal_median_min != 'NO'
""",
con,
)

,equal_median_min
0,ZN
1,CHAS


**Вывод:**
Равенство медианы и минимального значения, может значить следующее:

1) признак редкий

2) много отсутсвующих значений, которые заполнили минимумом


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [10]:
pd.read_sql(
    """
with chip as (
select
    '25' as build_counts,
    avg(RM) avg_rm_chip
from (select * from boston order by MEDV limit 25)
union all 
select
    '50' as build_counts,
    avg(RM) avg_rm_chip
from (select * from boston order by MEDV limit 50)
union all 
select
    '100' as build_counts,
    avg(RM) avg_rm_chip
from (select * from boston order by MEDV limit 100)
union all 
select
    '200' as build_counts,
    avg(RM) avg_rm_chip
from (select * from boston order by MEDV limit 200)
union all 
select
    '300' as build_counts,
    avg(RM) avg_rm_chip
from (select * from boston order by MEDV limit 300)
),
expensive as (
select
    '25' as build_counts,
    avg(RM) avg_rm_expensive
from (select * from boston order by MEDV desc limit 25)
union all 
select
    '50' as build_counts,
    avg(RM) avg_rm_expensive
from (select * from boston order by MEDV desc limit 50)
union all 
select
    '100' as build_counts,
    avg(RM) avg_rm_expensive
from (select * from boston order by MEDV desc limit 100)
union all 
select
    '200' as build_counts,
    avg(RM) avg_rm_expensive
from (select * from boston order by MEDV desc limit 200)
union all 
select
    '300' as build_counts,
    avg(RM) avg_rm_expensive
from (select * from boston order by MEDV desc limit 300)
)
select 
    c.build_counts,
    c.avg_rm_chip,
    e.avg_rm_expensive,
    e.avg_rm_expensive - c.avg_rm_chip as difference
from chip c
join expensive e using(build_counts)
""",
    con,
)

,build_counts,avg_rm_chip,avg_rm_expensive,difference
0,25,5.747840,7.637320,1.889480
1,50,5.753240,7.487060,1.733820
2,100,5.887120,7.198760,1.311640
3,200,5.911705,6.758680,0.846975
4,300,5.972227,6.539007,0.566780


**Вывод:** наболее сильная разница между средним количеством комнат видна среди 25ти самых дорогих и 25 самых дешевых домов. Даже в 100 самых дорогих домах в среднем на 1 комнату больше чем в 100 самых дешевых домах

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [11]:
pd.read_sql(
    """
with rank_t as (
select
    LSTAT,
    MEDV,
    rank() over (order by LSTAT) as rank
from boston
    )
select
    *
from rank_t
where medv == (select max(medv) from boston)
    """,
    con,
)

,LSTAT,MEDV,rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


**Вывод:** В самых дорогих домах низкий процент людей с более низким статусом

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [12]:
pd.read_sql(
    """
select 
    'граничит',
    avg(MEDV)
from boston
where CHAS = 1
union all
select 
    'не граничит',
    avg(MEDV)
from boston
where CHAS = 0
 """,
    con,
)

,'граничит',avg(MEDV)
0,граничит,28.440000
1,не граничит,22.093843


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [13]:
pd.read_sql(
"""
with avgdiff as (
select
    case when avg(CRIM) < (select avg(CRIM) from boston where CHAS = 1) then 'CRIM' else 'NO' end avgdiffs
from boston
union all
select
    case when avg(ZN) < (select avg(ZN) from boston where CHAS = 1) then 'ZN' else 'NO' end avgdiffs
from boston
union all
select
    case when avg(INDUS) < (select avg(INDUS) from boston where CHAS = 1) then 'INDUS' else 'NO' end avgdiffs
from boston
union all
select
    case when avg(CHAS) < (select avg(CHAS) from boston where CHAS = 1) then 'CHAS' else 'NO' end avgdiffs
from boston
union all
select
    case when avg(NOX) < (select avg(NOX) from boston where CHAS = 1) then 'NOX' else 'NO' end avgdiffs
from boston)
select
    *
from avgdiff
where avgdiffs != 'NO'
""",
con,
)

,avgdiffs
0,INDUS
1,CHAS
2,NOX


**Выводы:** INDUS - у реки больше объектов неторгового бизнеса, например - фабрики, склады и т.д. NOX - у реки высокая концентрация оксида азота, говорит о том, что у реки много объектов загрязняющих воздух (ТЭС, Металлургические предприятия и т.д.)

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [14]:
pd.read_sql(
    """
    select
        percentile * 10 as percentile,
        avg(INDUS),
        avg(NOX)
    from (
            select
                ntile(10) over (order by INDUS, NOX) as percentile,
                INDUS,
                NOX
            from boston
        )
    group by 1
    """,
    con,
)

,percentile,avg(INDUS),avg(NOX)
0,10,2.016078,0.438378
1,20,3.645294,0.494129
2,30,5.246275,0.448235
3,40,6.471765,0.470839
4,50,8.256078,0.531510
5,60,10.749216,0.515176
6,70,17.245800,0.564920
7,80,18.100000,0.684880
8,90,18.307200,0.714620
9,100,21.938800,0.692860


**Выводы:** Мы можем заметить, что между долей пром. предприятий и концентрацией оксида азота есть взаимосвязь: чем больше промышленных предприятий тем хуже загрязнен воздух. Однако мы можем заметить, что не везде увеличение промышленных предприятий увеличивает долю оксида азота. Это объясняется тем, что не все пром. объекты загрязняют воздух и в каких-то районах он лучше из-за низкой доли объектов сильно влияющих на оксид азота